In [3]:
import cv2

# Load the video
video_path = "myVideo.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the video writer to save the output video with silhouettes
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(
    'output_with_silhouette.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Load a pre-trained face classifier (using Haar cascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale (needed for face detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw the silhouette lines for each detected face
    for (x, y, w, h) in faces:
        # Calculate key points for silhouette lines
        center_x = x + w // 2
        neck_y = y + h  # Approximate neck position below the face
        # Approximate position of eyes (1/3 from top of face)
        eyes_y = y + h // 3

        # Draw vertical line for neck and nose
        cv2.line(frame, (center_x, y), (center_x, neck_y),
                 (0, 0, 0), thickness=2)

        # Draw horizontal line for eyes
        cv2.line(frame, (x, eyes_y), (x + w, eyes_y), (0, 0, 0), thickness=2)

    # Show the final frame
    cv2.imshow('Video with Silhouette', frame)

    # Write the frame to the output video
    output_video.write(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
output_video.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Initialize MediaPipe Pose and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between two vectors


def calculate_angle(a, b, c):
    # Points are given as (x, y) - Calculate the angle at point b
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Vector BA and BC
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle using the dot product formula
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Function to transform points to simulate a side view (perpendicular transformation)


def simulate_perpendicular_view(landmarks, width, height, side="left"):
    transformed_points = []

    # Side view: all x-coordinates should be set to the same value for a perpendicular effect
    shear_factor = 0.3 if side == "left" else -0.3
    # Reference x-coordinate to align all points for the perpendicular view
    ref_x = int(width / 2)

    for landmark in landmarks:
        # Convert normalized coordinates to pixel coordinates
        x = int(landmark.x * width)
        y = int(landmark.y * height)

        # Apply a shear transformation for a basic simulation of side view, and set the x to ref_x
        # Adjust x slightly to add a depth effect for better visualization
        new_x = ref_x + int(shear_factor * y)
        new_y = y

        transformed_points.append((new_x, new_y))

    return transformed_points


# Load the input video (change 'myVideo.mp4' to your video path or use 0 for webcam)
cap = cv2.VideoCapture('myVideo.mp4')

# Get the video frame dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
out = cv2.VideoWriter('output_simulated_perpendicular_views.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'), fps, (3 * width, height))

# Initialize MediaPipe Pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to capture frame")
            break

        # Convert the image to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image to detect pose landmarks
        results = pose.process(image_rgb)

        # Create a blank image for left and right side views
        left_view = np.zeros_like(frame)
        right_view = np.zeros_like(frame)

        # Check if landmarks are detected
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Generate silhouette for the central view (overlayed on the real frame)
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Extract specific landmark positions for neck and shoulder calculations
            NOSE_INDEX = 0
            LEFT_SHOULDER_INDEX = 11
            RIGHT_SHOULDER_INDEX = 12

            # Get landmark positions in pixels
            nose = (int(landmarks[NOSE_INDEX].x * width),
                    int(landmarks[NOSE_INDEX].y * height))
            left_shoulder = (int(landmarks[LEFT_SHOULDER_INDEX].x * width),
                             int(landmarks[LEFT_SHOULDER_INDEX].y * height))
            right_shoulder = (int(landmarks[RIGHT_SHOULDER_INDEX].x * width),
                              int(landmarks[RIGHT_SHOULDER_INDEX].y * height))

            # Draw neckline from nose to midpoint between shoulders
            neck_midpoint = (
                (left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
            cv2.line(frame, nose, neck_midpoint, (0, 0, 255), thickness=2)

            # Draw shoulder line between left and right shoulder
            cv2.line(frame, left_shoulder, right_shoulder,
                     (0, 255, 0), thickness=2)

            # Calculate and display the angle formed by the neck and shoulder line
            neck_angle = calculate_angle(
                left_shoulder, neck_midpoint, right_shoulder)
            cv2.putText(frame, f'Angle: {int(neck_angle)} deg', (neck_midpoint[0] + 10, neck_midpoint[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Simulate left and right side views using transformations
            left_side_points = simulate_perpendicular_view(
                landmarks, width, height, side="left")
            right_side_points = simulate_perpendicular_view(
                landmarks, width, height, side="right")

            # Draw left side silhouette
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                # In a side view, lines like shoulders should be a point
                # Shoulder and hip lines
                if (start_idx, end_idx) in [(11, 12), (23, 24)]:
                    start_coords = left_side_points[start_idx]
                    cv2.circle(left_view, start_coords, radius=5,
                               color=(255, 255, 255), thickness=-1)
                else:
                    start_coords = left_side_points[start_idx]
                    end_coords = left_side_points[end_idx]
                    cv2.line(left_view, start_coords,
                             end_coords, (255, 255, 255), 5)

            # Draw right side silhouette
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                # In a side view, lines like shoulders should be a point
                # Shoulder and hip lines
                if (start_idx, end_idx) in [(11, 12), (23, 24)]:
                    start_coords = right_side_points[start_idx]
                    cv2.circle(right_view, start_coords, radius=5,
                               color=(255, 255, 255), thickness=-1)
                else:
                    start_coords = right_side_points[start_idx]
                    end_coords = right_side_points[end_idx]
                    cv2.line(right_view, start_coords,
                             end_coords, (255, 255, 255), 5)

        # Concatenate central, left, and right views
        combined_view = np.hstack((left_view, frame, right_view))

        # Show the combined view
        cv2.imshow('Simulated Perpendicular Views', combined_view)

        # Write the combined view to the output video
        out.write(combined_view)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Initialize MediaPipe Pose and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between two vectors


def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Function to transform points to simulate a side view (perpendicular transformation)


def simulate_perpendicular_view(landmarks, width, height, side="left"):
    transformed_points = []
    shear_factor = 0.3 if side == "left" else -0.3
    ref_x = int(width / 2)

    for landmark in landmarks:
        x = int(landmark.x * width)
        y = int(landmark.y * height)
        new_x = ref_x + int(shear_factor * y)
        new_y = y
        transformed_points.append((new_x, new_y))

    return transformed_points


# Load the input video (change 'myVideo.mp4' to your video path or use 0 for webcam)
cap = cv2.VideoCapture('myVideo.mp4')

# Get the video frame dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video at 65% of original size
output_width = int(3 * width * 0.65)
output_height = int(height * 0.65)
out = cv2.VideoWriter('output_simulated_perpendicular_views.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'), fps, (output_width, output_height))

# Initialize MediaPipe Pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to capture frame")
            break

        # Convert the image to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image to detect pose landmarks
        results = pose.process(image_rgb)

        # Create a blank image for left and right side views
        left_view = np.zeros_like(frame)
        right_view = np.zeros_like(frame)

        # Check if landmarks are detected
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Generate silhouette for the central view (overlayed on the real frame)
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Extract specific landmark positions for neck and shoulder calculations
            NOSE_INDEX = 0
            LEFT_SHOULDER_INDEX = 11
            RIGHT_SHOULDER_INDEX = 12

            # Get landmark positions in pixels
            nose = (int(landmarks[NOSE_INDEX].x * width),
                    int(landmarks[NOSE_INDEX].y * height))
            left_shoulder = (int(landmarks[LEFT_SHOULDER_INDEX].x * width),
                             int(landmarks[LEFT_SHOULDER_INDEX].y * height))
            right_shoulder = (int(landmarks[RIGHT_SHOULDER_INDEX].x * width),
                              int(landmarks[RIGHT_SHOULDER_INDEX].y * height))

            # Draw neckline from nose to midpoint between shoulders
            neck_midpoint = (
                (left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
            cv2.line(frame, nose, neck_midpoint, (0, 0, 255), thickness=2)

            # Draw shoulder line between left and right shoulder
            cv2.line(frame, left_shoulder, right_shoulder,
                     (0, 255, 0), thickness=2)

            # Calculate and display the angle formed by the neck and shoulder line
            neck_angle = calculate_angle(
                left_shoulder, neck_midpoint, right_shoulder)
            cv2.putText(frame, f'Angle: {int(neck_angle)} deg', (neck_midpoint[0] + 10, neck_midpoint[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Simulate left and right side views using transformations
            left_side_points = simulate_perpendicular_view(
                landmarks, width, height, side="left")
            right_side_points = simulate_perpendicular_view(
                landmarks, width, height, side="right")

            # Draw left and right side views including the neckline movement
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                start_coords_left = left_side_points[start_idx]
                end_coords_left = left_side_points[end_idx]
                start_coords_right = right_side_points[start_idx]
                end_coords_right = right_side_points[end_idx]

                # Adjust neckline based on head movement for lateral views
                if start_idx == NOSE_INDEX and (end_idx == LEFT_SHOULDER_INDEX or end_idx == RIGHT_SHOULDER_INDEX):
                    neck_offset = (
                        neck_midpoint[0] - nose[0]) if end_idx == LEFT_SHOULDER_INDEX else -(neck_midpoint[0] - nose[0])
                    new_neck_x_left = left_side_points[NOSE_INDEX][0] + \
                        neck_offset
                    new_neck_x_right = right_side_points[NOSE_INDEX][0] - neck_offset

                    # Draw the neck in the lateral views with appropriate shift
                    cv2.line(left_view, (new_neck_x_left,
                             neck_midpoint[1]), left_side_points[NOSE_INDEX], (255, 255, 255), thickness=2)
                    cv2.line(right_view, (new_neck_x_right,
                             neck_midpoint[1]), right_side_points[NOSE_INDEX], (255, 255, 255), thickness=2)
                else:
                    # For normal connections, draw the silhouette lines
                    cv2.line(left_view, start_coords_left,
                             end_coords_left, (255, 255, 255), 5)
                    cv2.line(right_view, start_coords_right,
                             end_coords_right, (255, 255, 255), 5)

        # Concatenate central, left, and right views
        combined_view = np.hstack((left_view, frame, right_view))

        # Resize combined view to 65% of original size
        combined_view_resized = cv2.resize(
            combined_view, (output_width, output_height), interpolation=cv2.INTER_AREA)

        # Show the combined view
        cv2.imshow('Simulated Perpendicular Views', combined_view_resized)

        # Write the resized combined view to the output video
        out.write(combined_view_resized)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the midpoint between two points


def calculate_midpoint(point1, point2):
    return ((point1[0] + point2[0]) // 2, (point1[1] + point2[1]) // 2)

# Function to simulate side view points by rotating landmarks


def rotate_landmarks(landmarks, width, height, angle_degrees):
    rotated_points = []
    angle_radians = np.deg2rad(angle_degrees)
    for landmark in landmarks:
        x = landmark.x * width
        y = landmark.y * height
        rotated_x = x * np.cos(angle_radians) - y * np.sin(angle_radians)
        rotated_y = x * np.sin(angle_radians) + y * np.cos(angle_radians)
        rotated_points.append((int(rotated_x), int(rotated_y)))
    return rotated_points

# Function to add neck and face lines to the lateral views


def add_vertical_lines(view, neck_midpoint, nose_central, flip):
    # Add neck line
    neck_line_end = (neck_midpoint[0], neck_midpoint[1] -
                     (neck_midpoint[1] - nose_central[1]) * flip)
    cv2.line(view, neck_midpoint, neck_line_end, (255, 255, 255), 5)
    # Add face line
    face_line_end = (nose_central[0], neck_midpoint[1] -
                     (neck_midpoint[1] - nose_central[1]) * flip)
    cv2.line(view, nose_central, face_line_end, (255, 0, 0), 5)

# Function to add facial features to lateral views


def add_facial_features(view, side_points):
    left_eye = side_points[mp_pose.PoseLandmark.LEFT_EYE.value]
    right_eye = side_points[mp_pose.PoseLandmark.RIGHT_EYE.value]
    mouth_left = side_points[mp_pose.PoseLandmark.MOUTH_LEFT.value]
    mouth_right = side_points[mp_pose.PoseLandmark.MOUTH_RIGHT.value]

    # Drawing eyes
    cv2.circle(view, left_eye, 5, (255, 255, 255), -1)
    cv2.circle(view, right_eye, 5, (255, 255, 255), -1)

    # Drawing mouth
    cv2.line(view, mouth_left, mouth_right, (255, 255, 255), 5)


# Load the input video (change 'myVideo.mp4' to your video path or use 0 for webcam)
cap = cv2.VideoCapture('myVideo.mp4')

# Get the video frame dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Resize dimensions to 65%
resize_width = int(width * 0.65)
resize_height = int(height * 0.65)

# Define the codec and create a VideoWriter object to save the output video
out = cv2.VideoWriter('output_simulated_side_views.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'), fps, (3 * resize_width, resize_height))

# Initialize MediaPipe Pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to capture frame")
            break

        frame = cv2.resize(frame, (resize_width, resize_height))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        left_view = np.zeros_like(frame)
        right_view = np.zeros_like(frame)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            left_shoulder_central = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * resize_width),
                                     int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * resize_height))
            right_shoulder_central = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * resize_width),
                                      int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * resize_height))
            neck_midpoint_central = calculate_midpoint(
                left_shoulder_central, right_shoulder_central)
            nose_central = (int(landmarks[mp_pose.PoseLandmark.NOSE.value].x * resize_width),
                            int(landmarks[mp_pose.PoseLandmark.NOSE.value].y * resize_height))

            # Draw the vertical neck line behind the head
            cv2.line(frame, neck_midpoint_central,
                     nose_central, (255, 255, 255), 5)

            left_side_points = rotate_landmarks(
                landmarks, resize_width, resize_height, 90)
            right_side_points = rotate_landmarks(
                landmarks, resize_width, resize_height, -90)

            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]
                start_coords = left_side_points[start_idx]
                end_coords = left_side_points[end_idx]
                cv2.line(left_view, start_coords,
                         end_coords, (255, 255, 255), 5)

                start_coords = right_side_points[start_idx]
                end_coords = right_side_points[end_idx]
                cv2.line(right_view, start_coords,
                         end_coords, (255, 255, 255), 5)

            # Add facial features to the lateral views
            add_facial_features(left_view, left_side_points)
            add_facial_features(right_view, right_side_points)

            # Add neck and face lines to the lateral views with correct inclination
            add_vertical_lines(
                left_view, neck_midpoint_central, nose_central, 1)
            add_vertical_lines(
                right_view, neck_midpoint_central, nose_central, -1)

        combined_view = np.hstack((left_view, frame, right_view))
        cv2.imshow('Simulated Side Views', combined_view)
        out.write(combined_view)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()

In [5]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.optimize import minimize

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2,
                    enable_segmentation=False, min_detection_confidence=0.5)

# Function to process each frame and estimate the 2D keypoints


def process_frame(frame):
    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    # Extract 2D keypoints if available
    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            keypoints.append(
                (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])))
    return keypoints

# Optional: Function to optimize and estimate 3D joint angles


def optimize_3d_pose(keypoints_2d):
    # Placeholder optimization code (you would use a 3D model here)
    def objective_function(params):
        # This function would calculate the difference between the 2D projection of a 3D model
        # and the detected 2D keypoints.
        # `params` would represent the 3D joint angles or positions.
        # For simplicity, let's assume a mockup function that returns a constant error.
        return np.sum((params - np.array([1, 2, 3])) ** 2)

    # Initial guess for optimization
    initial_guess = np.zeros(3)

    # Run optimization to minimize the objective function
    result = minimize(objective_function, initial_guess, method='BFGS')
    # Return optimized 3D parameters (joint angles or positions)
    return result.x


# Video processing
input_video_path = 'myvideo.mp4'
output_video_path = 'output_with_pose_estimation.mp4'

# Open video capture and writer
cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps,
                      (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame to get 2D keypoints
    keypoints = process_frame(frame)

    # Draw keypoints and lines on the frame
    if keypoints:
        for x, y in keypoints:
            # Green circles for keypoints
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        # Draw lines between keypoints (simplified connections for visualization)
        # Add more connections as needed
        skeleton_connections = [(11, 12), (11, 23), (12, 24), (23, 24)]
        for pt1, pt2 in skeleton_connections:
            if pt1 < len(keypoints) and pt2 < len(keypoints):
                # Blue lines for connections
                cv2.line(frame, keypoints[pt1], keypoints[pt2], (255, 0, 0), 2)

    # Optionally, optimize 3D pose estimation based on 2D keypoints
    optimized_params = optimize_3d_pose(keypoints) if keypoints else None

    # Write frame to output video
    out.write(frame)

    # Display the frame with overlay (optional)
    cv2.imshow('Pose Estimation', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. The output video is saved as:", output_video_path)

c:\Users\stefa\anaconda3\envs\pyimage\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing complete. The output video is saved as: output_with_pose_estimation.mp4


In [7]:
import cv2
import mediapipe as mp
import numpy as np
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2,
                    enable_segmentation=False, min_detection_confidence=0.5)

# Function to process each frame and estimate the 2D keypoints


def process_frame(frame):
    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    # Extract 2D keypoints if available
    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            keypoints.append(
                (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])))
    return keypoints

# Function to calculate the angle between two points (e.g., eyes)


def calculate_angle(point1, point2):
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle = math.degrees(math.atan2(dy, dx))
    return angle

# Function to draw the head silhouette (two blue and two red lines)


def draw_head_silhouette(frame, keypoints, angle):
    # Define the center of the face and line lengths
    LEFT_EYE_INDEX = 1  # Adjust index based on landmark layout
    RIGHT_EYE_INDEX = 2  # Adjust index based on landmark layout

    if len(keypoints) > max(LEFT_EYE_INDEX, RIGHT_EYE_INDEX):
        # Get the coordinates of the eyes
        eye_left = keypoints[LEFT_EYE_INDEX]
        eye_right = keypoints[RIGHT_EYE_INDEX]

        # Calculate the center between the eyes
        face_center_x = (eye_left[0] + eye_right[0]) // 2
        face_center_y = (eye_left[1] + eye_right[1]) // 2
        head_line_length = 50
        neck_line_length = 40

        # Calculate angles for the two blue lines (head sides)
        left_angle = angle - 30
        right_angle = angle + 30

        # Draw left blue line
        left_end_x = int(face_center_x + head_line_length *
                         math.cos(math.radians(left_angle)))
        left_end_y = int(face_center_y + head_line_length *
                         math.sin(math.radians(left_angle)))
        cv2.line(frame, (face_center_x, face_center_y),
                 (left_end_x, left_end_y), (255, 0, 0), 2)

        # Draw right blue line
        right_end_x = int(face_center_x + head_line_length *
                          math.cos(math.radians(right_angle)))
        right_end_y = int(face_center_y + head_line_length *
                          math.sin(math.radians(right_angle)))
        cv2.line(frame, (face_center_x, face_center_y),
                 (right_end_x, right_end_y), (255, 0, 0), 2)

        # Draw vertical red lines (neck) below each blue line end
        # Left red line (neck)
        neck_left_end_y = left_end_y + neck_line_length
        cv2.line(frame, (left_end_x, left_end_y),
                 (left_end_x, neck_left_end_y), (0, 0, 255), 2)

        # Right red line (neck)
        neck_right_end_y = right_end_y + neck_line_length
        cv2.line(frame, (right_end_x, right_end_y),
                 (right_end_x, neck_right_end_y), (0, 0, 255), 2)

# Function to draw the vertical line on the face


def draw_face_vertical_line(frame, keypoints, angle):
    LEFT_EYE_INDEX = 1
    RIGHT_EYE_INDEX = 2

    if len(keypoints) > max(LEFT_EYE_INDEX, RIGHT_EYE_INDEX):
        # Get the coordinates of the eyes
        eye_left = keypoints[LEFT_EYE_INDEX]
        eye_right = keypoints[RIGHT_EYE_INDEX]

        # Determine the center point between the eyes
        face_center_x = (eye_left[0] + eye_right[0]) // 2
        face_center_y = (eye_left[1] + eye_right[1]) // 2

        # Length of the vertical line
        line_length = 50

        # Calculate start and end points for the line based on inclination
        line_end_x = int(face_center_x + line_length *
                         math.cos(math.radians(angle)))
        line_end_y = int(face_center_y + line_length *
                         math.sin(math.radians(angle)))
        line_start_x = int(face_center_x - line_length *
                           math.cos(math.radians(angle)))
        line_start_y = int(face_center_y - line_length *
                           math.sin(math.radians(angle)))

        # Draw the vertical face line
        cv2.line(frame, (line_start_x, line_start_y), (line_end_x,
                 line_end_y), (0, 255, 0), 2)  # Green color for face line


# Video processing
input_video_path = 'myvideo.mp4'
output_video_path = 'output_with_silhouette_and_face_line.mp4'

# Open video capture and writer
cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps,
                      (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame to get 2D keypoints
    keypoints = process_frame(frame)

    # Check if we have both eyes to calculate the inclination angle
    if len(keypoints) >= 3:
        # Get eye positions and calculate head tilt angle
        eye_left = keypoints[1]
        eye_right = keypoints[2]
        angle = calculate_angle(eye_left, eye_right)

        # Draw the main head silhouette (blue and red lines)
        draw_head_silhouette(frame, keypoints, angle)

        # Draw the vertical line on the face
        draw_face_vertical_line(frame, keypoints, angle)

    # Write frame to output video
    out.write(frame)

    # Display the frame with overlay (optional)
    cv2.imshow('Silhouette with Face Line', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. The output video is saved as:", output_video_path)

Processing complete. The output video is saved as: output_with_silhouette_and_face_line.mp4
